# Watson OpenScale用にトランザクションをセットする方法

In [ ]:
# Replace with your WML Credentials
wml_credentials={

  "apikey": "xxxxxxxxxx",
  "iam_apikey_description": "xxxxxxxxxx",
  "iam_apikey_name": "xxxxxxxxxx",
  "iam_role_crn": "xxxxxxxxxx",
  "iam_serviceid_crn": "xxxxxxxxxx",
  "instance_id": "xxxxxxxxxx",
  "password": "xxxxxxxxxx",
  "url": "xxxxxxxxxx",
  "username": "xxxxxxxxxx"

}

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
# scoring url はMachine Learningの管理機能で取得したものを利用します。
scoring_url = 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/xxxxxxxx/deployments/xxxxxxx/online'

# 呼出し時のパラメータ
params = {
  "fields": ['NA','K','AGE','CHOLESTEROL','SEX','BP'],
  "values": [[ 0.66,0.05,43.0,'HIGH','M','LOW']]
}

# 1. スコアリング・エンドポイントをWML-Pythonクライアントから使う場合
## client.deployments.score(scoring_url, params, transaction_id="XXX")で指定

### [WML Python Document - deployments - score](https://wml-api-pyclient.mybluemix.net/#deployments)

In [ ]:
#モデル呼出し時のパラメーターでtransaction_id=を指定
scores = client.deployments.score(scoring_url, params, transaction_id="CUSTOMER-001-01")
# 結果表示
print(scores)

# 2. スコアリング・エンドポイントをREST(POST)で使う場合
#   HTTPヘダーのX-Global-Transaction-Idで指定

### [AIOS Explaining transactions](https://cloud.ibm.com/docs/services/ai-openscale/insight-explain.html#view)

In [ ]:
#  AI OpenScaleのinternalDBの接続情報の入手
import requests, json

# IAM Tokenの入手
def getToken(apikey):
    url = 'https://iam.bluemix.net/identity/token'
    header = {'Content-Type': 'application/x-www-form-urlencoded', 'Accept': 'application/json'}
    payload = {'grant_type': 'urn:ibm:params:oauth:grant-type:apikey', 'apikey': apikey}
    response = requests.post(url, payload, headers=header).json()
    return response['access_token']
##  CHANGE ME
##  platform_apikey ... IBM Cloudの 管理 - セキュリティ - プラットフォームAPIキー より作成
platform_apikey = "xxxxxxxxxx"
header = {'Content-Type': 'application/json', 'Accept':'application/json',
    'Authorization': 'Bearer ' + getToken(platform_apikey), 'X-Global-Transaction-Id':'CUSTOMER-002-01'}
response = requests.post(scoring_url, data=json.dumps(params), headers=header).json()
print(response)
